# Lead Scoring Model

#### IMPORTING STATEMENTS

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import re
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel("Data_Science_Internship.xlsx")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46608 entries, 0 to 46607
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0      46608 non-null  int64         
 1   Agent_id        46608 non-null  object        
 2   status          46608 non-null  object        
 3   lost_reason     43244 non-null  object        
 4   budget          42908 non-null  object        
 5   lease           44267 non-null  object        
 6   movein          32970 non-null  datetime64[ns]
 7   source          46608 non-null  object        
 8   source_city     46608 non-null  object        
 9   source_country  46608 non-null  object        
 10  utm_source      46608 non-null  object        
 11  utm_medium      46608 non-null  object        
 12  des_city        46608 non-null  object        
 13  des_country     46608 non-null  object        
 14  room_type       23061 non-null  object        
 15  le

In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)


In [5]:
df.drop(df[(df["status"] != "LOST") & (df["status"] != "WON")].index, inplace = True)
df.replace('9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0', 0, inplace=True)
df.lease.replace('lease', 0, inplace=True)
df.replace(0, np.nan, inplace=True)
df.dropna(thresh= 12, inplace= True)

***PREPROCESSING ON LEASE COLUMN***

In [ ]:
df["lease"] = df["lease"].str.replace("\\", " ")
df["lease"] = df["lease"].str.replace("/", " ")
df["lease"] = df["lease"].str.replace("mnth", "months")
df["lease"] = df["lease"].str.replace("Stay", "")
df["lease"] = df["lease"].str.replace("weeeks", "")
df["lease"] = df["lease"].str.replace("2 years", "100")
df["lease"] = df["lease"].str.replace("Other", "0", flags= re.IGNORECASE)
df["lease"] = df["lease"].str.replace("44 51", "45")
df["lease"] = df["lease"].str.replace("WEEKS", "weeks")
df["lease"] = df["lease"].str.replace("not sure", "0")
df["lease"] = df["lease"].str.replace("ASAP", "0")
df["lease"] = df["lease"].str.replace("3 years", "150")
df["lease"] = df["lease"].str.replace("1 month ", "4")
df["lease"] = df["lease"].str.replace("<=8", "8")
df["lease"] = df["lease"].str.replace("7 days", "1")
df["lease"] = df["lease"].str.replace("44 or 51 ", "45")
df["lease"] = df["lease"].str.replace("51s", "51")
df["lease"] = df["lease"].str.replace("Weeks", "week")
df["lease"] = df["lease"].str.replace("43 44 ", "44")
df.loc[df["lease"].str.contains("Full year",na= False, flags= re.IGNORECASE), "lease"] = "42 weeks"
df.loc[df["lease"].str.contains("1 year",na= False, flags= re.IGNORECASE), "lease"] = "42 weeks"
df.loc[df["lease"].str.contains("1year",na= False, flags= re.IGNORECASE), "lease"] = "42 weeks"
df.loc[df["lease"].str.contains("Full yyear",na= False, flags= re.IGNORECASE), "lease"] = "42 weeks"
df.loc[df["lease"].str.contains("Full-year",na= False, flags= re.IGNORECASE), "lease"] = "42 weeks"
df.loc[df["lease"].str.contains("Full-COURSE",na= False, flags= re.IGNORECASE), "lease"] = "52 weeks"
df.loc[df["lease"].str.contains("COMPLETE ",na= False, flags= re.IGNORECASE), "lease"] = "52 weeks"
df.loc[df["lease"].str.contains("SEM",na= False, flags= re.IGNORECASE), "lease"] = "22 weeks"
df.loc[df["lease"].str.contains("SHORT",na= False, flags= re.IGNORECASE), "lease"] = "9 weeks"
df.loc[df["lease"].str.contains("dec",na= False, flags= re.IGNORECASE), "lease"] = "52 weeks"
df.loc[df["lease"].str.contains("dec",na= False, flags= re.IGNORECASE), "lease"] = "52 weeks"
df.loc[df["lease"].str.contains("whole year",na= False, flags= re.IGNORECASE), "lease"] = "42 weeks"
df.loc[df["lease"].str.contains("full course",na= False, flags= re.IGNORECASE), "lease"] = "52 weeks"
df.loc[df["lease"].str.contains("academic",na= False, flags= re.IGNORECASE), "lease"] = "52 weeks"
df.loc[df["lease"].str.contains("aug",na= False, flags= re.IGNORECASE), "lease"] = "32 weeks"



In [ ]:
b = df[df["lease"].str.endswith(("month","mnth","months","m"), na=False)]
b[["start", "end"]] = b["lease"].str.split(" ", expand=True)
b = b.drop("end", axis=1)
b["start"] = b["start"].str.replace("long", "12")
b["start"] = b["start"].str.replace("pm","")
b["start"] = b["start"].str.replace("months","")
b["start"] = b["start"].str.replace("m","")
b["start"] = b["start"].str.replace("-", "+")


In [ ]:
for i in b["start"].index:
    if "+" in b["start"][i]:
        j = b["start"][i]
        l,m = j.split("+")
        b["start"][i]= (int(l)+int(m))/2

In [ ]:
b["lease"] = (b["start"].astype(int)) * 4
b["lease"] = b["lease"].replace(2100, 0)
b.drop("start", axis=1 , inplace=True)
b["lease"] = b["lease"].astype(str)
df.update(b)
df = df.reset_index(drop=True)


In [ ]:
df.lease.iloc[34527] = "12"
df.lease.iloc[9806] = "4"
df.lease.iloc[27381] = "51"
df.lease.iloc[30763] = "44"
df.lease.iloc[12166] = "0"
df.lease.iloc[11862] = "20"

In [ ]:
df.lease = df.lease.astype(str)

In [ ]:
for i in df.lease.index:
    if "week" in df.lease.iloc[i] or "weeks" in df.lease.iloc[i]:
        j = df["lease"].iloc[i]
        l,m = j.split("w",1)
        df["lease"][i]= (l)
    elif df.lease.iloc[i] == "":
        df.lease.iloc[i] = "0"
    

In [ ]:
for i in df.lease.index:
    if "-" in df.lease.iloc[i]:
        j = df["lease"].iloc[i]
        l,m = j.split("-",1)
        df["lease"][i]= (int(l)+int(m))/2

***CONVERTING LEASE COLUMN INTO CATEGORICAL COLUMN***

In [ ]:
df.lease = df["lease"].replace("nan", "0")
df.lease = df.lease.astype(float)
df.lease = df.lease.astype(int)
df["cat_lease"] = df["lease"].replace({range(0,10):"lowest",range(10,20):"low",range(20,30):"average",range(30,40):"medium", range(40,50):"high",range(50,70):"higher", range(70, 300): "highest"})
df.cat_lease = le.fit_transform(df.cat_lease)

***PREPROCESSING ON BUDGET COLUMN***

In [ ]:
try:
    df["budget"].iloc[18738] = "0"
    df.loc[df["budget"].str.contains("no",na= False, flags= re.IGNORECASE), "budget"] = "0"
    df["budget"] = df["budget"].str.replace("pw", " Per Week")
    df["budget"] = df["budget"].str.replace("pcm", " per month")
    df["budget"] = df["budget"].str.replace("PCM", " per month")
    df["budget"] = df["budget"].str.replace("pm", " per month")
    df["budget"] = df["budget"].str.replace("  Per Week -151 ", "-151")
    df["budget"] = df["budget"].str.replace("0 month", "0 per month")
    df["budget"] = df["budget"].str.replace("0 week", "0 Per Week")
    df["budget"] = df["budget"].str.replace(" Per Week", "")
    df["budget"] = df["budget"].str.replace(" Per week", "")
    df["budget"] = df["budget"].str.replace(" per week ", "")
    df["budget"] = df["budget"].str.replace(" per weeks", "")
    df["budget"] = df["budget"].str.replace("per ek", " Per Week")
    df["budget"] = df["budget"].str.replace("Per week", "")
    df["budget"] = df["budget"].str.replace(" per week", "")
    df["budget"] = df["budget"].str.replace("£", "")
    df["budget"] = df["budget"].str.replace("0-0", "0")
    df["budget"] = df["budget"].str.replace("/", " ")
    df["budget"] = df["budget"].replace(np.nan, "0")
    df["budget"] = df["budget"].str.replace("month ", " per month")
    df["budget"] = df.budget.str.replace("160 200", "160-200")
    df["budget"] = df.budget.str.replace(">", "")
    df["budget"] = df.budget.str.replace("0-100 Pounds", "0-113")
    df["budget"] = df.budget.str.replace("0-200-400", "200")
    df["budget"] = df.budget.str.replace("€", "")
    df["budget"] = df.budget.str.replace("s", "")
    df["budget"] = df.budget.str.replace("Le than  ", "")
    df["budget"] = df.budget.str.replace("0-60 to 75", "70")
    df["budget"] = df.budget.str.replace("0-200-260", "200-260")
    df["budget"] = df.budget.str.replace("Around 400 pound", "450")
    df["budget"] = df.budget.str.replace("prw", "")
    df["budget"] = df.budget.str.replace("0-10,000 GBP PA", "0-235")
    df["budget"] = df.budget.str.replace("cheapet-highet", "0 - 100")
    df["budget"] = df.budget.str.replace("0-260-275", "270")
    df["budget"] = df.budget.str.replace("50 -60 gbp", "55-65")
    df["budget"] = df.budget.str.replace("0-120 140", "130")
    df["budget"] = df.budget.str.replace("0-120 to 180", "150")
    df["budget"] = df.budget.str.replace("0-5-10 lakh", "115-230")
    df["budget"] = df.budget.str.replace("90-100 gbp", "110")
    df["budget"] = df.budget.str.replace("max budget", "")
    df["budget"] = df.budget.str.replace("0-130-175", "0-175")
    df["budget"] = df.budget.str.replace("flexible", "100")
    df["budget"] = df.budget.str.replace("maximum", "")
    df["budget"] = df.budget.str.replace("0-Between", "")
    df["budget"] = df.budget.str.replace("110 s", "110")
    df["budget"] = df.budget.str.replace("flexible", "100")
    df["budget"] = df.budget.str.replace(" week", "")
    df["budget"] = df.budget.str.replace(" WEEK", "")
    df["budget"] = df.budget.str.replace(" W", "")
    df["budget"] = df.budget.str.replace("week", "")
    df["budget"] = df.budget.str.replace("Pereek", "")
    df["budget"] = df.budget.str.replace("7000 pound", "165")
    df["budget"] = df.budget.str.replace("month.", "month")
    df["budget"] = df.budget.str.replace("momth", "month")
    df["budget"].iloc[18008] = "0"
    df.budget.iloc[610] = "0"
    df.room_type.iloc[610] = "Ensuite"
    df["budget"] = df['budget'].replace(np.nan, 0)
except Exception as e:
    print(e)

In [ ]:
try:
    a = df[df["budget"].str.endswith("month", na=False)]
    a[["start", "end"]] = a["budget"].str.split("per month", expand=True)
except Exception as e:
    print(e)

In [ ]:
try:    
    a["start"] = a["start"].str.replace("500 600", "500-600")
    a["start"] = a["start"].str.replace(">", "0-")
    a[["start", "end"]] = a["start"].str.split("month", expand=True,n=1)
    a[["start", "middle", "last"]] = a["start"].str.split("-", expand=True)
    a["start"] = a["start"].replace(np.nan, 0)
    a["middle"] = a["middle"].str.replace("Per","")
    a["start"] = a["start"].str.replace("p","")
    a["start"] = a["start"].str.replace("a","")
    a["start"] = a["start"].str.replace("Per","")
    a["start"] = a["start"].str.replace("m","")
    a["middle"] = a["middle"].replace(np.nan, 0)
    a["start"] = (a["start"]).astype(int)
    a["middle"] = (a["middle"]).astype(int)
    a["start"] = (a["start"]/4)
    a["middle"] = (a["middle"]/4)
except Exception as e:
    print(e)

In [ ]:
a = a.drop("end", axis=1)
a = a.drop("last", axis=1)

In [ ]:
 
def concat_col_mid(data: pd.DataFrame):
    try:
        start_col = data['start']
        middle_col = data['middle']
        
        for row in data.index:
            start_value = start_col[row]
            middle_col_val = middle_col[row]
            
            new_value = f"{int(start_value)}-{int(middle_col_val)}" if not int(middle_col_val) == 0 else str(int(start_value))
            data["budget"][row] = new_value
        
        return data
    except Exception as e:
        print(e)
    
a = concat_col_mid(a)
    
    

In [ ]:
for i in a["budget"].index:
    if "-" in a["budget"][i]:
        j = a["budget"][i]
        l,m = j.split("-")
        a["budget"][i]= (int(l)+int(m))/2

In [ ]:
for i in df["budget"].index:
    if "-" in df["budget"][i] and "month" not in df["budget"][i]:
        j = df["budget"][i]
        l,m = j.split("-")
        df["budget"][i]= (int(l)+int(m))/2
    elif "pound" in df["budget"].iloc[i]:
        j = df["budget"][i]
        l,m = j.split("p")
        df["budget"][i]= int(float(l) * 1.13)

In [ ]:
df.budget = df.budget.astype(str)
df.update(a)


In [ ]:

df["room_type"].iloc[22428]  = "Studio"
df["budget"].iloc[22428] = "0"


In [ ]:
def changed_budget(df):    
    df.budget = df.budget.str.replace("EPW", "")
    df.budget = df.budget.str.replace("300 per ek", "300")
    df.budget = df.budget.str.replace("PW", "")
    df.budget = df.budget.str.replace("euro", "")
    df.budget = df.budget.str.replace("460  per month", "115")
    df.budget = df.budget.str.replace('1000 per  per monthfor 3', "85")
    df.budget = df.budget.str.replace("400 m", "100")
    df.budget = df.budget.str.replace("600 Per monthly", "150")
    df.budget = df.budget.str.replace("500  per month", "125")
    df.budget = df.budget.str.replace("44 51", "45")
    df.budget = df.budget.str.replace("max", "")
    df.budget = df.budget.str.replace("tbh", "0")
    df.budget = df.budget.str.replace("165 170", "165")
    df.budget = df.budget.str.replace("300ly", "300")
    df.budget = df.budget.str.replace("100 Gbpo", "110")
    df.budget = df.budget.str.replace("tbd", "0")
    df.budget = df.budget.str.replace("Be30 200 GBP", "128")
    df.budget = df.budget.str.replace("low", "30")
    df.budget = df.budget.str.replace("30et", "30")
    df.budget = df.budget.str.replace("100 p for 2 people", "50")
    df.budget = df.budget.str.replace("okay with", "0")
    df.budget = df.budget.str.replace("0 250", "125")
    df.budget = df.budget.str.replace("na", "0")
    df.budget = df.budget.str.replace("na", "0")
    df.budget = df.budget.str.replace("\\", "")
    df.budget = df.budget.str.replace("PPW", "")
    df.budget = df.budget.str.replace("150$", "135")
    df.budget = df.budget.str.replace("460 month max", "115")
    df.budget = df.budget.str.replace("w", "")
    df.budget = df.budget.str.replace("200 to 210", "205")
    df.budget = df.budget.str.replace("700PPM", "175")
    df.budget = df.budget.str.replace("1000 per month for 3", "85")
    df.budget = df.budget.str.replace("cheapet", "0")
    df.budget = df.budget.str.replace("500 PM", "125")
    df.budget = df.budget.str.replace("150'", "150")
    df.budget = df.budget.str.replace("280W", "280")
    df.budget = df.budget.str.replace("P", "")
    df.budget = df.budget.str.replace("Be30 200 GB", "128")
    df.budget = df.budget.str.replace("300 per ek", "300")
    df.budget = df.budget.str.replace("be30 100 gbp", "72")
    df.budget = df.budget.str.replace("(30 budget)", "")

    return df


In [ ]:
df = changed_budget(df)
df.budget = df.budget.astype(float)
df.budget = df.budget.replace(np.nan, 0)
df.budget = df.budget.astype(int)


***CONVERTING BUDGET COLUMN INTO CATEGORICAL COLUMN***

In [ ]:
df["cat_budget"] = df["budget"].replace({range(0,100):"lowest",range(100,200):"low",range(200,300):"average",range(300,400):"medium", range(400,500):"good",range(500,700):"better", range(700, 1600): "best"})
df.cat_budget = le.fit_transform(df.cat_budget)


In [ ]:
df.reset_index(inplace= True, drop= True)

##### LABEL ENCODING CATEGORICAL COLUMNS

In [ ]:
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O' and feature not in ["lease"] and feature not in ["budget"]]
for i in df.columns:
        if i in categorical_features:
            df[i] = le.fit_transform(df[i])

***DROPPING SOME COLUMNS***

In [ ]:
df = df.drop(["movein","budget", "lease"],axis=1)

***Saving dataframes to .csv files***

In [ ]:
df.to_csv("cleaned_data.csv",index= False)


In [ ]:
X = df.drop(["status"],axis=1)
y = df["status"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
train_data = pd.concat([X_train,y_train], axis=1)
test_data = pd.concat([X_test,y_test], axis=1)
train_data.to_csv("train.csv",index= False)
test_data.to_csv("test.csv",index= False)
